In [4]:
import aiohttp
import asyncio
from bs4 import BeautifulSoup
import pandas as pd
import re
from tqdm import tqdm
import nest_asyncio
import random

# 既存のイベントループを再利用するために必要
nest_asyncio.apply()

all_data = []
semaphore = asyncio.Semaphore(10)  # 同時に実行されるタスクの数を10に制限

user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3', 
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36',
    # 他のユーザーエージェントを追加
]

async def fetch(session, url):
    async with semaphore:
        headers = {'User-Agent': random.choice(user_agents)}
        async with session.get(url, headers=headers) as response:
            await asyncio.sleep(random.uniform(1, 3))  # ランダムな待機時間を追加
            return await response.text()

async def fetch_detail(session, job_id):
    detail_url = f'https://doda.jp/DodaFront/View/JobSearchDetail/j_jid__{job_id}/-tab__jd/-fm__jobdetail/-mpsc_sid__10/-tp__1/'
    async with semaphore:
        headers = {'User-Agent': random.choice(user_agents)}
        async with session.get(detail_url, headers=headers) as response:
            await asyncio.sleep(random.uniform(1, 3))  # ランダムな待機時間を追加
            return await response.text()

def get_text_from_selectors(soup, selectors):
    for selector in selectors:
        element = soup.select_one(selector)
        if element:
            return element.get_text(strip=True, separator='\n')
    return ''

async def process_page(session, page_number, pbar):
    try:
        url = f'https://doda.jp/DodaFront/View/JobSearchList.action?pic=1&ds=0&oc=03L%2C14L&so=50&ind=0101S%2C0103S%2C0107S&op=17&pf=0&ne=3%2C4&tp=1&page={page_number}&prsrt=1'
        print(f"Fetching page {page_number}: {url}")  # デバッグ用の出力
        html = await fetch(session, url)
        soup = BeautifulSoup(html, 'html.parser')
        job_elements = soup.find_all('h2', class_='title clrFix')

        for job_element in job_elements:
            company_span = job_element.find('span', class_='company width688')
            company_name = company_span.get_text(separator=" ", strip=True).replace('NEW', '').strip()

            job_title = job_element.find('span', class_='job width688').get_text(strip=True)
            job_url = job_element.find('a', class_='_JobListToDetail')['href']

            pattern = r'j_jid__(\d+)'
            match = re.search(pattern, job_url)
            job_id = match.group(1)

            # 対象と給与を取得
            target = job_element.find_next('dt', string='対象').find_next_sibling('dd').get_text(strip=True)
            salary = job_element.find_next('dt', string='給与').find_next_sibling('dd').get_text(strip=True)

            # 詳細ページに移動して追加情報を取得
            detail_html = await fetch_detail(session, job_id)
            detail_soup = BeautifulSoup(detail_html, 'html.parser')

            # 業種
            li_elements = detail_soup.find('ul', class_='clrFix').find_all('li')
            industry = li_elements[2].get_text(strip=True)
            if industry.endswith('>'):
                industry = industry[:-1].strip()

            # 連絡先の中身を取得
            contact_info = get_text_from_selectors(detail_soup, [
                'dt:-soup-contains("連絡先") + dd',
                'th:-soup-contains("連絡先") + td'
            ])

            # 所在地を取得
            location = get_text_from_selectors(detail_soup, [
                'th:-soup-contains("所在地") + td',
                'dt:-soup-contains("所在地") + dd'
            ])

            # 代表者を取得
            representative = get_text_from_selectors(detail_soup, [
                'th:-soup-contains("代表者") + td',
                'dt:-soup-contains("代表者") + dd'
            ])

            # 企業URLを取得
            company_url = ''
            for selector in ['th:-soup-contains("企業URL") + td a', 'dt:-soup-contains("企業URL") + dd a']:
                element = detail_soup.select_one(selector)
                if element:
                    company_url = element['href']
                    break

            # データをリストに追加
            all_data.append({
                '会社名': company_name,
                '求人タイトル': job_title,
                '詳細ページURL': job_url,
                '対象': target,
                '給与': salary,
                '求人ID': job_id,
                '業種': industry,
                '連絡先': contact_info,
                '所在地': location,
                '代表者': representative,
                '企業URL': company_url
            })
        pbar.update(1)  # ページの処理が完了したら進捗バーを更新
    except Exception:
        pass

async def main():
    async with aiohttp.ClientSession() as session:
        for start_page in range(1, 481, 10):
            tasks = []
            with tqdm(total=10, desc=f"Processing pages {start_page} to {start_page + 9}") as pbar:
                for page_number in range(start_page, start_page + 10):
                    task = process_page(session, page_number, pbar)
                    tasks.append(task)
                await asyncio.gather(*tasks)

# 非同期イベントループを実行
loop = asyncio.get_event_loop()
loop.run_until_complete(main())

# データフレームに変換
df = pd.DataFrame(all_data)
df.to_csv('doda-scraping.csv', index=False, encoding='utf-8')

Processing pages 1 to 10:   0%|          | 0/10 [00:00<?, ?it/s]

Fetching page 1: https://doda.jp/DodaFront/View/JobSearchList.action?pic=1&ds=0&oc=03L%2C14L&so=50&ind=0101S%2C0103S%2C0107S&op=17&pf=0&ne=3%2C4&tp=1&page=1&prsrt=1
Fetching page 2: https://doda.jp/DodaFront/View/JobSearchList.action?pic=1&ds=0&oc=03L%2C14L&so=50&ind=0101S%2C0103S%2C0107S&op=17&pf=0&ne=3%2C4&tp=1&page=2&prsrt=1
Fetching page 3: https://doda.jp/DodaFront/View/JobSearchList.action?pic=1&ds=0&oc=03L%2C14L&so=50&ind=0101S%2C0103S%2C0107S&op=17&pf=0&ne=3%2C4&tp=1&page=3&prsrt=1
Fetching page 4: https://doda.jp/DodaFront/View/JobSearchList.action?pic=1&ds=0&oc=03L%2C14L&so=50&ind=0101S%2C0103S%2C0107S&op=17&pf=0&ne=3%2C4&tp=1&page=4&prsrt=1
Fetching page 5: https://doda.jp/DodaFront/View/JobSearchList.action?pic=1&ds=0&oc=03L%2C14L&so=50&ind=0101S%2C0103S%2C0107S&op=17&pf=0&ne=3%2C4&tp=1&page=5&prsrt=1
Fetching page 6: https://doda.jp/DodaFront/View/JobSearchList.action?pic=1&ds=0&oc=03L%2C14L&so=50&ind=0101S%2C0103S%2C0107S&op=17&pf=0&ne=3%2C4&tp=1&page=6&prsrt=1
Fetching p

Processing pages 11 to 20:   0%|          | 0/10 [00:00<?, ?it/s]

Fetching page 11: https://doda.jp/DodaFront/View/JobSearchList.action?pic=1&ds=0&oc=03L%2C14L&so=50&ind=0101S%2C0103S%2C0107S&op=17&pf=0&ne=3%2C4&tp=1&page=11&prsrt=1
Fetching page 12: https://doda.jp/DodaFront/View/JobSearchList.action?pic=1&ds=0&oc=03L%2C14L&so=50&ind=0101S%2C0103S%2C0107S&op=17&pf=0&ne=3%2C4&tp=1&page=12&prsrt=1
Fetching page 13: https://doda.jp/DodaFront/View/JobSearchList.action?pic=1&ds=0&oc=03L%2C14L&so=50&ind=0101S%2C0103S%2C0107S&op=17&pf=0&ne=3%2C4&tp=1&page=13&prsrt=1
Fetching page 14: https://doda.jp/DodaFront/View/JobSearchList.action?pic=1&ds=0&oc=03L%2C14L&so=50&ind=0101S%2C0103S%2C0107S&op=17&pf=0&ne=3%2C4&tp=1&page=14&prsrt=1
Fetching page 15: https://doda.jp/DodaFront/View/JobSearchList.action?pic=1&ds=0&oc=03L%2C14L&so=50&ind=0101S%2C0103S%2C0107S&op=17&pf=0&ne=3%2C4&tp=1&page=15&prsrt=1
Fetching page 16: https://doda.jp/DodaFront/View/JobSearchList.action?pic=1&ds=0&oc=03L%2C14L&so=50&ind=0101S%2C0103S%2C0107S&op=17&pf=0&ne=3%2C4&tp=1&page=16&prsrt=

Processing pages 21 to 30:   0%|          | 0/10 [00:00<?, ?it/s]

...

Processing pages 461 to 470:   0%|          | 0/10 [00:00<?, ?it/s]

Fetching page 461: https://doda.jp/DodaFront/View/JobSearchList.action?pic=1&ds=0&oc=03L%2C14L&so=50&ind=0101S%2C0103S%2C0107S&op=17&pf=0&ne=3%2C4&tp=1&page=461&prsrt=1
Fetching page 462: https://doda.jp/DodaFront/View/JobSearchList.action?pic=1&ds=0&oc=03L%2C14L&so=50&ind=0101S%2C0103S%2C0107S&op=17&pf=0&ne=3%2C4&tp=1&page=462&prsrt=1
Fetching page 463: https://doda.jp/DodaFront/View/JobSearchList.action?pic=1&ds=0&oc=03L%2C14L&so=50&ind=0101S%2C0103S%2C0107S&op=17&pf=0&ne=3%2C4&tp=1&page=463&prsrt=1
Fetching page 464: https://doda.jp/DodaFront/View/JobSearchList.action?pic=1&ds=0&oc=03L%2C14L&so=50&ind=0101S%2C0103S%2C0107S&op=17&pf=0&ne=3%2C4&tp=1&page=464&prsrt=1
Fetching page 465: https://doda.jp/DodaFront/View/JobSearchList.action?pic=1&ds=0&oc=03L%2C14L&so=50&ind=0101S%2C0103S%2C0107S&op=17&pf=0&ne=3%2C4&tp=1&page=465&prsrt=1
Fetching page 466: https://doda.jp/DodaFront/View/JobSearchList.action?pic=1&ds=0&oc=03L%2C14L&so=50&ind=0101S%2C0103S%2C0107S&op=17&pf=0&ne=3%2C4&tp=1&pag

Processing pages 471 to 480:   0%|          | 0/10 [00:00<?, ?it/s]

Fetching page 471: https://doda.jp/DodaFront/View/JobSearchList.action?pic=1&ds=0&oc=03L%2C14L&so=50&ind=0101S%2C0103S%2C0107S&op=17&pf=0&ne=3%2C4&tp=1&page=471&prsrt=1
Fetching page 472: https://doda.jp/DodaFront/View/JobSearchList.action?pic=1&ds=0&oc=03L%2C14L&so=50&ind=0101S%2C0103S%2C0107S&op=17&pf=0&ne=3%2C4&tp=1&page=472&prsrt=1
Fetching page 473: https://doda.jp/DodaFront/View/JobSearchList.action?pic=1&ds=0&oc=03L%2C14L&so=50&ind=0101S%2C0103S%2C0107S&op=17&pf=0&ne=3%2C4&tp=1&page=473&prsrt=1
Fetching page 474: https://doda.jp/DodaFront/View/JobSearchList.action?pic=1&ds=0&oc=03L%2C14L&so=50&ind=0101S%2C0103S%2C0107S&op=17&pf=0&ne=3%2C4&tp=1&page=474&prsrt=1
Fetching page 475: https://doda.jp/DodaFront/View/JobSearchList.action?pic=1&ds=0&oc=03L%2C14L&so=50&ind=0101S%2C0103S%2C0107S&op=17&pf=0&ne=3%2C4&tp=1&page=475&prsrt=1
Fetching page 476: https://doda.jp/DodaFront/View/JobSearchList.action?pic=1&ds=0&oc=03L%2C14L&so=50&ind=0101S%2C0103S%2C0107S&op=17&pf=0&ne=3%2C4&tp=1&pag

Processing pages 471 to 480: 100%|██████████| 10/10 [02:42<00:00, 16.25s/it]
